In [7]:
import numpy as np

class FCLayer:
    def __init__(self, input_size, output_size):
        self.w = np.random.random((output_size, input_size)) - 0.5
        self.b = np.zeros((output_size, 1))
        self.layer_input = None

    def forward(self, layer_input):
        self.layer_input = layer_input
        return np.dot(self.w, layer_input) + self.b

    def backprop(self, layer_error, learning_rate=0.01):
        m = layer_error.shape[1]
        self.w -= np.dot(layer_error, self.layer_input.T) / m * learning_rate
        self.b -= np.sum(layer_error, axis=1, keepdims=True) / m * learning_rate
        return np.dot(self.w.T, layer_error)

class ActivationLayer:
    def __init__(self, func, func_prime):
        self.func = func
        self.func_prime = func_prime
        self.layer_input = None

    def forward(self, layer_input):
        self.layer_input = layer_input
        return self.func(layer_input)

    def backprop(self, layer_error, learning_rate=0.01):
        return self.func_prime(self.layer_input) * layer_error

class NeuralNetwork:
    def __init__(self, layers, loss, loss_prime, learning_rate=0.01):
        self.layers = layers
        self.loss = loss
        self.loss_prime = loss_prime
        self.learning_rate = learning_rate

    def forward(self, x):
        res = [x]
        for layer in self.layers:
            res.append(layer.forward(res[-1]))
        return res[-1]

    def fit(self, x, y, epochs=100, verbose=100):
        for i in range(epochs):
            ret = self.forward(x)
            loss = self.loss(ret, y)
            res = [self.loss_prime(ret, y)]
            for layer in reversed(self.layers):
                res.append(layer.backprop(res[-1], learning_rate=self.learning_rate))
            if i % verbose == 0:
                print(f'Loss at epoch {i}: {loss}')

In [8]:
relu = lambda x: np.maximum(0, x)
drelu = lambda x: 1.0 * (x > 0)

sigmoid = lambda x: 1 / (1 + np.exp(-x))
dsigmoid = lambda x: sigmoid(x) * (1 - sigmoid(x))

tanh = lambda x: np.tanh(x)
dtanh = lambda x: 1 - np.tanh(x) ** 2

In [9]:
nn = NeuralNetwork(
    layers=[
        FCLayer(2, 5),
        ActivationLayer(relu, drelu),
        FCLayer(5, 1),
        ActivationLayer(sigmoid, dsigmoid)
    ],
    loss=lambda p, y: np.sum((p - y) ** 2) / p.shape[1],
    loss_prime=lambda p, y: 2 * (p - y) / p.shape[1],
    learning_rate=0.1
)
x_train = np.asarray([0, 0, 1, 1, 0, 1, 0, 1]).reshape((2, 4))
y_train = np.asarray([0, 1, 1, 0])
nn.fit(x_train, y_train, epochs=10000, verbose=5000)
nn.forward(x_train)

Loss at epoch 0: 0.24476246106013447
Loss at epoch 5000: 0.16769471696304059


array([[0.33521771, 0.33521771, 0.96548215, 0.33521771]])

In [10]:
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt

def as_categorical(x):
    t = np.zeros((10, 1))
    t[x] = 1
    return t

digits = load_digits()
TRAIN_CNT = 1200
x_train = digits.images[:TRAIN_CNT].reshape((TRAIN_CNT, 8 * 8)).T
x_train = x_train.astype('float32')
x_train /= 255
y_train = np.asarray([as_categorical(target) for target in digits.target[:TRAIN_CNT]]).T.reshape((10, TRAIN_CNT))

EVAL_CNT = 500
x_test = digits.images[TRAIN_CNT:TRAIN_CNT + EVAL_CNT].reshape((EVAL_CNT, 8 * 8)).T
x_test = x_test.astype('float32')
x_test /= 255
y_test = np.asarray([as_categorical(target) for target in digits.target[TRAIN_CNT:TRAIN_CNT + EVAL_CNT]]).T.reshape((10, EVAL_CNT))

nn = NeuralNetwork(
    layers=[
        FCLayer(64, 32),
        ActivationLayer(relu, drelu),
        FCLayer(32, 16),
        ActivationLayer(tanh, dtanh),
        FCLayer(16, 10),
        ActivationLayer(sigmoid, dsigmoid)
    ],
    loss=lambda p, y: np.sum((p - y) ** 2) / p.shape[1],
    loss_prime=lambda p, y: 2 * (p - y) / p.shape[1],
    learning_rate=0.1
)
nn.fit(x_train, y_train, epochs=100000, verbose=5000)

Loss at epoch 0: 2.4869365038325086
Loss at epoch 5000: 1.9670505372355482
Loss at epoch 10000: 1.5904675230242018
Loss at epoch 15000: 1.327894275116316
Loss at epoch 20000: 1.1626121408364356
Loss at epoch 25000: 1.0645831912047448
Loss at epoch 30000: 1.0066036115727266
Loss at epoch 35000: 0.9713648206547794
Loss at epoch 40000: 0.9491370076100081
Loss at epoch 45000: 0.9345829473161235
Loss at epoch 50000: 0.9247312722934252
Loss at epoch 55000: 0.9178624424001155
Loss at epoch 60000: 0.9129542334532686
Loss at epoch 65000: 0.9093685243166052
Loss at epoch 70000: 0.9066986103722459
Loss at epoch 75000: 0.9046775812353098
Loss at epoch 80000: 0.9031264906399851
Loss at epoch 85000: 0.9019208693450499
Loss at epoch 90000: 0.900971948122769
Loss at epoch 95000: 0.9002171831395587
